In [1]:
from utils import imagine_nav_planner
import numpy as np
import matplotlib.pyplot as plt
from utils import fmm_planner
from utils import raycast
%cd analysis
agent_position = [280,275]
occupancy = np.load("occupancy_sample.npy")
traversible = np.load("traversible_sample.npy")
frontiers = np.load("frontiers_sample.npy")
%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring


/workspace/Projects/SG-VLN-axi/analysis


In [4]:
def approximate_tangents(path: np.ndarray, normalize: bool = True) -> np.ndarray:
    """
    Approximates the tangent vector at each coordinate of a path.

    The tangent at point P_i is approximated as:
    - P_1 - P_0 for the first point (forward difference)
    - P_N-1 - P_N-2 for the last point (N-1) (backward difference)
    - P_i+1 - P_i-1 for interior points P_i (central difference)

    Args:
        path (np.ndarray): An Nx2 NumPy array of (x, y) coordinates.
        normalize (bool): If True, normalize the tangent vectors to unit length.
                          Defaults to True.

    Returns:
        np.ndarray: An Nx2 NumPy array of tangent vectors (dx, dy).
                    If a tangent vector's magnitude is close to zero (e.g., duplicate points),
                    and normalize is True, it will be returned as [0, 0].
                    Returns an empty array of shape (0,2) if input path has < 1 point.
                    Returns [[0,0]] if input path has exactly 1 point.
    """
    N = path.shape[0]

    if N == 0:
        return np.empty((0, 2), dtype=path.dtype)
    if N == 1:
        # Tangent is undefined for a single point, return [0,0] as a convention
        return np.array([[0.0, 0.0]], dtype=path.dtype)

    tangents = np.zeros_like(path, dtype=float) # Use float for potential normalization

    # First point: forward difference
    tangents[0] = path[1] - path[0]

    # Last point: backward difference
    if N > 1: # This check is technically redundant due to N==1 case above, but good for clarity
        tangents[N - 1] = path[N - 1] - path[N - 2]

    # Interior points: central difference
    # path[2:] gives elements from index 2 to end
    # path[:-2] gives elements from index 0 up to (but not including) N-2
    # So, path[2:] - path[:-2] effectively computes P[i+1] - P[i-1] for i in [1, ..., N-2]
    if N > 2:
        tangents[1:-1] = path[2:] - path[:-2]
        
    # Handle N=2 case explicitly if central diff wasn't run,
    # though the initial forward/backward handles it.
    # For N=2, tangents[0] = path[1]-path[0], tangents[1] = path[1]-path[0]
    # This is consistent with the above logic.
    return tangents
def get_headings(path):
    tangents = approximate_tangents(path)
    return np.arctan2(tangents[:,1],tangents[:,0])


In [ ]:
planner = fmm_planner.FMMPlanner(traversible)

import skimage
n=5
selem = skimage.morphology.disk(5)
goal_map = np.zeros_like(traversible)

goal_map[frontiers[n][0],frontiers[n][1]]=1
goal = skimage.morphology.binary_dilation(goal_map, selem)
planner.set_multi_goal(goal)

path = [agent_position]
state = agent_position

for i in range(20):
    stg_x, stg_y, replan, stop = planner.get_short_term_goal(state)
    state = [stg_x , stg_y]
    path.append(state)
    if stop:
        print("stopping")
        break
path = np.array(path)
print(path)


from time import time


planner.set_goal(agent_position)
fmm_dist = planner.fmm_dist

t0 = time()
step_cost = fmm_dist[path[:,0].astype(int),path[:,1].astype(int)]
gamma = np.power(0.99,step_cost)

mask,prob = raycast.get_path_exploration(path,(occupancy==1),(occupancy==0),theta=3.1,n_rays=20,gamma=gamma,p=1/100000)

distance_weight = np.exp(-(fmm_dist*0.01))  #divide by 100-200 seems reasonable
mask*=distance_weight
total_probability = 1-np.prod(1-mask)

print("probability: "+str(total_probability))
print(time()-t0)


plt.clf()
plt.plot(path[:,1],path[:,0],lw=0.1,c='w',marker='.')
#plt.imshow(distance_weight)
plt.imshow(traversible+goal_map+mask*300000)
plt.scatter(agent_position[1],agent_position[0],s=2,c='g')
plt.scatter(frontiers[n][1],frontiers[n][0],s=2,c='r')
plt.savefig("myfig")


stopping
[[280. 275.]
 [283. 278.]
 [286. 281.]
 [290. 282.]
 [293. 285.]
 [296. 288.]
 [299. 290.]
 [303. 291.]
 [306. 294.]
 [309. 297.]
 [310. 301.]
 [313. 304.]
 [314. 308.]
 [317. 311.]
 [318. 315.]
 [319. 319.]
 [320. 323.]]
probability: 0.15506553917538934
0.04654049873352051


In [7]:
fmm_dist[path[:,0].astype(int),path[:,1].astype(int)]

array([ 0.        ,  4.55895359,  8.7601914 , 12.4618982 , 16.64756159,
       20.85508274, 24.44323375, 28.25293804, 32.4333596 , 36.61775275,
       39.92183384, 44.15472509, 47.76511209, 51.98621654, 55.74054317,
       59.59710517, 63.51990387])

In [8]:
x,y = 150,200
w,h = 300,300
phi = np.ones((w,h))
phi[x,y]=0
import skfmm
distance = skfmm.distance(phi,dx=1)
plt.imshow(np.exp(-distance/30.))
plt.savefig("myfig")